In [ ]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

# Define the TriageAgent with custom logic
class TriageAgent(AssistantAgent):
    def __init__(self, name):
        super().__init__(
            name,
            llm_config=False,  # Disable LLM
            system_message="You are the Triage Agent. Analyze the user's message and decide which agent to route to."
        )

    def decide_next_agent(self, message):
        if "support" in message.lower():
            return "SupportAgent"
        elif "technical" in message.lower():
            return "TechnicalAgent"
        else:
            return "SupportAgent"  # Default to SupportAgent

    def generate_reply(self, messages=None, sender=None, **kwargs):
        # Extract the last message from the user
        print("triage message:",messages)
        if messages:
            last_message = messages[-1]["content"]
            next_agent = self.decide_next_agent(last_message)
            return f"Routing to {next_agent}."
        else:
            return "No messages to process. Please start the conversation."

# Define the SupportAgent with custom logic
class SupportAgent(AssistantAgent):
    def __init__(self, name):
        super().__init__(
            name,
            llm_config=False,  # Disable LLM
            system_message="You are the Support Agent. Provide helpful and friendly support for general inquiries."
        )

    def generate_reply(self, messages=None, sender=None, **kwargs):
        return "Thank you for contacting support. How can we assist you today?"

# Define the TechnicalAgent with custom logic
class TechnicalAgent(AssistantAgent):
    def __init__(self, name):
        super().__init__(
            name,
            llm_config=False,  # Disable LLM
            system_message="You are the Technical Agent. Provide detailed and technical assistance for complex issues."
        )

    def generate_reply(self, messages=None, sender=None, **kwargs):
        return "Thank you for contacting technical support. Please describe the issue you're facing."

# Define the UserProxyAgent
user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",  # Set to "NEVER" for fully automated chat
    code_execution_config=False
)

# Create instances of the agents
triage_agent = TriageAgent("TriageAgent")
support_agent = SupportAgent("SupportAgent")
technical_agent = TechnicalAgent("TechnicalAgent")

# Define the group chat and manager
groupchat = GroupChat(
    agents=[user_proxy, triage_agent,technical_agent, support_agent ],
    messages=[],
    max_round=10,
    speaker_selection_method="round_robin"  # Ensures the TriageAgent gets a turn
)

manager = GroupChatManager(groupchat=groupchat)

# Start the conversation
user_proxy.initiate_chat(
    manager,
    message="I need technical help with my account login. It's not working."
)

In [ ]:
import asyncio
import nest_asyncio
from typing import AsyncGenerator, List, Sequence, Tuple
from autogen_agentchat.base import Response
from autogen_agentchat.agents import AssistantAgent, BaseChatAgent
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_core import CancellationToken


async def main() -> None:
    def check_calculation(x: int, y: int, answer: int) -> str:
        if x + y == answer:
            return "Correct!"
        else:
            return "Incorrect!"
    
    class CalcAgent(BaseChatAgent):
        def __init__(self, name: str):
            super().__init__(name,name)

        @property
        def produced_message_types(self) -> Sequence[type[ChatMessage]]:
            return (TextMessage,)

        async def on_messages(self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken) -> Response:
            last_message = messages[-1].content.strip()
            count = int(last_message)
            return await self.process(count)

        async def process(self, count: int) -> Response:
            inner_messages: List[ChatMessage] = []
            final_msg = TextMessage(content="Done!", source=self.name)
            inner_messages.append(final_msg)
            print("final:",final_msg.content)
            return Response(chat_message=final_msg, inner_messages=inner_messages)

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        pass

    agent1 = AssistantAgent(
        "Agent1",
        model_client,
        description="For calculation",
        system_message="Calculate the sum of two numbers",
    )
    agent2 = AssistantAgent(
        "Agent2",
        model_client,
        tools=[check_calculation],
        description="For checking calculation",
        system_message="Check the answer and respond with 'Correct!' or 'Incorrect!'",
    )

    def selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
        if len(messages) == 1 or messages[-1].content == "Incorrect!":
            return "Agent1"
        if messages[-1].source == "Agent1":
            return "Agent2"
        return None

    termination = TextMentionTermination("Correct!")
    team = SelectorGroupChat(
        [agent1, agent2],
        model_client=model_client,
        selector_func=selector_func,
        termination_condition=termination,
    )

    await Console(team.run_stream(task="What is 1 + 1?"))

nest_asyncio.apply()
asyncio.run(main())

In [15]:
from autogen import UserProxyAgent, AssistantAgent, GroupChat

# Define agents (LLM disabled)
user_proxy = UserProxyAgent(
    name="User_proxy",
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    code_execution_config=False
)
agent1 = AssistantAgent(
    name="Agent1",
    llm_config=False,  # Empty config list to disable LLM
)
agent2 = AssistantAgent(
    name="Agent2",
    llm_config={"config_list": []},  # Empty config list to disable LLM
)

# Define a simple routing function
def route_message(message):
    if "keyword1" in message:
        return agent1
    elif "keyword2" in message:
        return agent2
    else:
        return None  # Or a default agent

# Start chat, manually routing messages
user_proxy.initiate_chat(agent1, message="Starting the conversation.") # Initial message

while True:
    user_input = user_proxy.get_human_input("Enter your message (or 'exit'): ")
    if user_input.lower() == "exit":
        break

    recipient = route_message(user_input)

    if recipient:
        user_proxy.send(user_input, recipient)
        response = recipient.receive()  # Manually get the response.
        if response:
            print(f"{recipient.name}: {response['content']}")
    else:
        print("No suitable agent found for this message.")

User_proxy (to User_proxy):

Starting the conversation. Enter your message:

--------------------------------------------------------------------------------
User_proxy (to User_proxy):

keyword1

--------------------------------------------------------------------------------


TypeError: ConversableAgent.get_human_input() missing 1 required positional argument: 'prompt'

In [18]:
import asyncio
import nest_asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import Swarm
from autogen_agentchat.conditions import MaxMessageTermination


async def main() -> None:
    #model_client = OpenAIChatCompletionClient(model="gpt-4o")

    agent1 = AssistantAgent(
        "Alice",
        llm,
        handoffs=["Bob"],
        system_message="You are Alice and you only answer questions about yourself.",
    )
    agent2 = AssistantAgent(
        "Bob", model_client=model_client, system_message="You are Bob and your birthday is on 1st January."
    )

    termination = MaxMessageTermination(3)
    team = Swarm([agent1, agent2], termination_condition=termination)

    stream = team.run_stream(task="What is bob's birthday?")
    async for message in stream:
        print(message)


nest_asyncio.apply()
asyncio.run(main())

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [22]:
import asyncio
from autogen_core import CancellationToken
from autogen_agentchat.agents import UserProxyAgent
from autogen_agentchat.messages import TextMessage


async def simple_user_agent():
    agent1 = UserProxyAgent("user_proxy")
    agent2 = UserProxyAgent("user_proxy")
    response = await asyncio.create_task(
        agent1.on_messages(
            [TextMessage(content="What is your name? ", source="user")],
            cancellation_token=CancellationToken(),
        )
    )
    print(f"1.Your name is {response.chat_message.content}")
    response = await asyncio.create_task(
        agent2.on_messages(
            [TextMessage(content="What is your name? ", source="user")],
            cancellation_token=CancellationToken(),
        )
    )
    print(f"2.Your name is {response.chat_message.content}")

nest_asyncio.apply()
asyncio.run(simple_user_agent())

1.Your name is s
2.Your name is n


: 